In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

In [ ]:
import os
file_path = "/content/drive/MyDrive/Colab Notebooks/movie_archive.zip"
if os.path.isfile(file_path):
    print(f"File exists: {file_path}")
else:
    print(f"File does not exist: {file_path}")

File exists: /content/drive/MyDrive/Colab Notebooks/movie_archive.zip


In [ ]:
#zip_file_path =
with zipfile.ZipFile(file_path) as zf:
    directories = zf.namelist()
    pos_dir = [subdir for subdir in directories if "pos" in subdir]
    neg_dir = [subdir for subdir in directories if "neg" in subdir]

        # if not file.endswith('.png'): # optional filtering by filetype
        #     continue
        # with zf.open(file) as f:
        #     image = pygame.image.load(f, namehint=file)

In [ ]:
print(pos_dir[0])

movie_reviews/movie_reviews/pos/cv000_29590.txt


In [ ]:
train_percent = 0.8
pos_train_dir = pos_dir[:int(train_percent*len(pos_dir))]
pos_test_dir = pos_dir[int(train_percent*len(pos_dir)):]

neg_train_dir = neg_dir[:int(train_percent*len(neg_dir))]
neg_test_dir = neg_dir[int(train_percent*len(neg_dir)):]

In [ ]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Example multi-line string
# multi_line_string = """
# <p>Hello, this is a sample text with <b>HTML</b> tags.</p>
# <p>Here are some numbers: 123, 456, 789.</p>
# <p>Another line with numbers and <a href="#">more HTML</a> tags.</p>
# """
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(f_path):
  with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/movie_archive.zip", 'r') as zip_ref:
    with zip_ref.open(f_path) as file:
      file_content = file.read().decode('utf-8').lower()
    #content = file_content.strip().split('\n')
    content = BeautifulSoup(file_content, "html.parser").text
    content = re.sub(r'\d+', '', content)
    content = ' '.join(content.split())
    tokens = word_tokenize(content)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  return lemmatized_tokens


# # 1. Split the string into lines
# lines = multi_line_string.strip().split('\n')

# # Function to remove HTML tags
# def remove_html_tags(text):
#     return BeautifulSoup(text, "html.parser").text

# # Function to remove numbers from a string
# def remove_numbers(text):
#     return re.sub(r'\d+', '', text)

# # Function to remove extra whitespaces
# def remove_extra_whitespaces(text):
#     return ' '.join(text.split())

# # Process each line to remove numbers, HTML tags, and extra whitespaces
# cleaned_lines = []
# for line in lines:
#     # Remove HTML tags
#     line_without_html = remove_html_tags(line)
#     # Remove numbers
#     line_no_numbers = remove_numbers(line_without_html)
#     # Remove extra whitespaces
#     line_cleaned = remove_extra_whitespaces(line_no_numbers)
#     cleaned_lines.append(line_cleaned)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#from gensim.models import KeyedVectors
# import gensim.downloader as api
from gensim.downloader import load

# Load a pre-trained Word2Vec model
word2vec_model = load('glove-wiki-gigaword-50')
# Load a pre-trained Word2Vec model from gensim's API
#word2vec_model = api.load("word2vec-google-news-300")
# Load the pre-trained Word2Vec model
# Replace 'path/to/GoogleNews-vectors-negative300.bin' with the actual path to your model file
# model_path = 'GoogleNews-vectors-negative300.bin'
# word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)



In [ ]:
def get_embeddings(token, model):
        if token in model:
            return model[token]
        else:
          return np.zeros(model.vector_size)

            # Handle out-of-vocabulary (OOV) words if needed

# def get_embeddings(tokens, model):
#     embeddings = {}
#     for token in tokens:
#         if token in model:
#             embeddings[token] = model[token]
#         else:
#             # Handle out-of-vocabulary (OOV) words if needed
#             continue #embeddings[token] = None
#     return embeddings

In [ ]:
import numpy as np
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
num_columns = 50
num_rows = 0  # No rows initially

# Create the csr_matrix
embed_matrix = csr_matrix((num_rows, num_columns))

for each_path in pos_train_dir:
  # with open(each_path, 'r') as file:
  #   text = file.read()
    cleaned_text = clean_text(each_path)
    num_of_tokens = len(cleaned_text)
    if num_of_tokens == 0:
      avg_file_embedding = np.zeros(num_columns)
    else:
      sum_embeddings = 0
      for token in cleaned_text:
        sum_embeddings += get_embeddings(token, word2vec_model)
    avg_file_embedding = sum_embeddings / num_of_tokens
    data_matrix = csr_matrix(avg_file_embedding).reshape(1,-1)

# Append the new data to the existing matrix
    embed_matrix = vstack([embed_matrix, data_matrix])


for each_path in neg_train_dir:
  # with open(each_path, 'r') as file:
  #   text = file.read()
    cleaned_text = clean_text(each_path)
    num_of_tokens = len(cleaned_text)
    if num_of_tokens == 0:
      avg_file_embedding = np.zeros(num_columns)
    else:
      sum_embeddings = 0
      for token in cleaned_text:
        sum_embeddings += get_embeddings(token, word2vec_model)
    avg_file_embedding = sum_embeddings / num_of_tokens
    ndata_matrix = csr_matrix(avg_file_embedding).reshape(1,-1)

# Append the new data to the existing matrix
    embed_matrix = vstack([embed_matrix, ndata_matrix])


In [ ]:
ones = np.ones(int(train_percent*len(pos_dir)))

# Step 2: Create an array of 0s with 800 elements
zeros = np.zeros(int(train_percent*len(pos_dir)))

# Step 3: Concatenate the two arrays
labels = np.concatenate((ones, zeros))

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix

# Assuming `embed_matrix` is a csr_matrix and `labels` is a NumPy array of labels
# embed_matrix = csr_matrix(...)  # Your CSR matrix of embeddings
# labels = np.array(...)  # Your labels (0 or 1 for binary classification)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if convergence issues occur
model.fit(embed_matrix, labels)

# Make predictions
# predictions = model.predict(embed_matrix)

LogisticRegression(max_iter=1000)

In [ ]:
nembed_matrix = csr_matrix((num_rows, num_columns))
for each_path in neg_test_dir:
  # with open(each_path, 'r') as file:
  #   text = file.read()
    cleaned_text = clean_text(each_path)
    num_of_tokens = len(cleaned_text)
    if num_of_tokens == 0:
      avg_file_embedding = np.zeros(num_columns)
    else:
      sum_embeddings = 0
      for token in cleaned_text:
        sum_embeddings += get_embeddings(token, word2vec_model)
    avg_file_embedding = sum_embeddings / num_of_tokens
    ndata_matrix = csr_matrix(avg_file_embedding).reshape(1,-1)

# Append the new data to the existing matrix
    nembed_matrix = vstack([nembed_matrix, ndata_matrix])

In [ ]:
predictions = model.predict(nembed_matrix)

In [ ]:
from collections import Counter

# Example list


# Use Counter to count occurrences
occurrences_dict = Counter(predictions)

print(occurrences_dict)

Counter({0.0: 149, 1.0: 51})
